<a href="https://colab.research.google.com/github/SubhashMOthukuri/QUANTUM-COMPUTING-LAB-BY-THE-CODING-SCHOOL/blob/main/Week6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 6: Implementing BB84 with Eve**
---

### **Description**
In this notebook, we will apply our programming and Cirq skills to implement the BB84 protocol.

<br>

### **Structure**
**Part 1**: [Alice and Bob Review](#p1)

**Part 2**: [Eve](#p2)


<br>

### **Resources**
* [BB84 Class Documentation](https://the-codingschool.github.io/bb84/documentation/bb84.html)
* [Cirq Basics Cheat Sheet](https://docs.google.com/document/d/1Ir1qtXS2-C_tGThk00P1fZfZjoC8oet4N9Rtkw7LgVY/edit?usp=drive_link)

<br>

**Before starting, run the code below to import all necessary functions and libraries.**


In [ ]:
# @title
import warnings
warnings.filterwarnings('ignore')


try:
    import cirq
except ImportError:
    print('installing cirq...')
    !pip install cirq --quiet
    import cirq
    print('installed cirq.')


#!git clone https://github.com/the-codingschool/bb84.git
#from bb84.bb84 import BB84

from cirq import NamedQubit, Simulator, Circuit

class BB84:

    """
    Implements the BB84 protocol for quantum key distribution.

    The BB84 protocol allows two parties, Alice and Bob, to generate a shared secret key, which can then be used
    for secure communication. The protocol also involves checking for the presence of an eavesdropper, Eve.
    This implementation provides a simulated environment for the protocol's execution, including the potential
    interception by Eve.


    <b>NOTE</b>:
        This class requires external definitions for the quantum circuits and simulator to function properly.

    <br>

    © 2024 The Coding School, All rights reserved.
    """

    qubit: NamedQubit
    """The qubit being used for the protocol. Must be defined externally, for example:
    ```python
    protocol.qubit = cirq.NamedQubit('q0')
    ```
    """

    simulator: Simulator
    """The quantum circuit simulator. Must be defined externally, for example:
    ```python
    protocol.simulator = cirq.Simulator()
    ```
    """

    alice_send_0_no_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 0 without an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_0_no_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_1_no_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 1 without an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_1_no_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_0_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 0 with an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_0_H_circuit = cirq.Circuit()
    ```
    """

    alice_send_1_H_circuit: Circuit
    """The quantum circuit that Alice will use to encode a 1 with an H gate. Must be defined externally, for example:
    ```python
    protocol.alice_send_1_H_circuit = cirq.Circuit()
    ```
    """

    eve_intercept_circuit: Circuit
    """The quantum circuit that Eve will use to intercept the qubit between Phase 1 and 2. Must be defined externally, for example:
    ```python
    protocol.eve_intercept_circuit = cirq.Circuit()
    ```
    """

    bob_receive_no_H_circuit: Circuit
    """The quantum circuit that Bob will use to receive and measure the qubit during Phase 3 without using an H gate. Must be defined externally, for example:
    ```python
    protocol.bob_receive_no_H_circuit = cirq.Circuit()
    ```
    """

    bob_receive_H_circuit: Circuit
    """The quantum circuit that Bob will use to receive and measure the qubit during Phase 3 using an H gate. Must be defined externally, for example:
    ```python
    protocol.bob_receive_H_circuit = cirq.Circuit()
    ```
    """


    eve_intercept: str
    """Indicates if Eve intercepts the qubits ('yes' or 'no'). Should be set upon initialization or after calling `reset()`."""

    alice_bit: int
    """The current bit Alice wants to send. Should only be modified indirectly through the `send_bit` method's parameters."""

    bob_bit: int
    """The bit received by Bob after measurement. Should only be modified indirectly through the `send_bit` method's parameters."""

    eve_bit: int
    """The bit measured by Eve if interception occurs. Should only be modified indirectly through the `send_bit` method's parameters."""

    does_alice_apply_H: str
    """Indicates if Alice applies the Hadamard gate ('yes' or 'no'). Should only be modified indirectly through the `send_bit` method's parameters."""

    does_bob_apply_H: str
    """Indicates if Bob applies the Hadamard gate ('yes' or 'no'). Should only be modified indirectly through the `send_bit` method's parameters."""


    alice_key: list
    """The secret key generated by Alice. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    bob_key: list
    """The secret key generated by Bob. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    eve_key: list
    """The key intercepted by Eve, if any. <b>SHOULD NOT BE MODIFIED BY USER</b>."""

    bit_num: int
    """Counter for the number of bits processed so far. <b>SHOULD NOT BE MODIFIED BY USER</b>."""


    def __init__(self, eve_intercept = 'no'):

      """
      Initializes the BB84 protocol simulation with default or specified settings.

      Parameters:
          `eve_intercept` (str, optional): Determines if Eve will attempt to intercept the qubits ('yes' or 'no').
                                         Defaults to 'no'.

      Example Usage:
      ```python
      protocol_without_eve = BB84()

      also_protocol_without_eve = BB84(eve_intercept = 'no')

      protocol_with_eve = BB84(eve_intercept = 'yes')
      ```
      """

      self.alice_bit = None
      self.bob_bit = None

      self.eve_bit = None
      self.eve_intercept = eve_intercept

      self.does_alice_apply_H = None
      self.does_bob_apply_H = None

      self.qubit = None
      self.simulator = None

      self.alice_send_0_no_H_circuit = None
      self.alice_send_1_no_H_circuit = None
      self.alice_send_0_H_circuit = None
      self.alice_send_1_H_circuit = None

      self.eve_intercept_circuit = None

      self.bob_receive_no_H_circuit = None
      self.bob_receive_H_circuit = None

      self.alice_key = []
      self.bob_key = []
      self.eve_key = []

      self.bit_num = 1


    def phase_1_circuit(self):

      """
      Creates a circuit such that Alice encodes her bit into a qubit and applies an H gate if she's chosen to do so.
      Then she sends this qubit through her quantum channel to Bob, hoping Eve does not intercept.

      <b>Returns</b>:
          The quantum circuit for Phase 1 of BB84 (through Alice sending her qubit).

      <b>INTERNAL USE ONLY</b>
      """

      if self.alice_bit == 0:
        if self.does_alice_apply_H == 'no':
          return self.alice_send_0_no_H_circuit
        else:
          return self.alice_send_0_H_circuit

      else:
        if self.does_alice_apply_H == 'no':
          return self.alice_send_1_no_H_circuit
        else:
          return self.alice_send_1_H_circuit


    def eve_circuit(self):

      """
      Creates a circuit for Eve's role in the protocol, which depends on if she's intercepting or not.

      <b>Returns</b>:
          The quantum circuit for Eve's interception if it occurs, otherwise an empty circuit.

      <b>INTERNAL USE ONLY</b>
      """

      if self.eve_intercept == 'yes':
        return self.eve_intercept_circuit
      else:
        return cirq.Circuit()


    def phase_2_circuit(self):

      """
      Creates a circuit such that Bob receives the qubit, applies an H gate if he's decided to do so,
      and then measures it.

      <b>Returns</b>:
          The quantum circuit for Phase 2 of BB84 (through Bob measuring the qubit).

      <b>INTERNAL USE ONLY</b>
      """

      if self.does_bob_apply_H == 'no':
        return self.bob_receive_no_H_circuit
      else:
        return self.bob_receive_H_circuit


    def restart(self):

      """
      Resets the protocol such that the circuits are all the same, but they are being used with blank keys and on a new quantum channel.
      """

      self.alice_key = []
      self.bob_key = []
      self.eve_key = []

      self.bit_num = 1


    def send_bit(self, alice_bit, does_alice_apply_H, does_bob_apply_H, compare_bits = 'no'):

      """
      Simulates the full BB84 protocol, potentially intercepted by Eve.

      Parameters:
        <ul>
          <li>`alice_bit` (int): The bit Alice wants to send.</li>
          <li>`does_alice_apply_H` (str): Indicates if Alice applies the Hadamard gate ('yes' or 'no').</li>
          <li>`does_bob_apply_H` (str): Indicates if Bob applies the Hadamard gate ('yes' or 'no').</li>
          <li>`compare_bit` (str, optional): Determines if Alice and Bob compare their bits directly ('yes' or 'no').
                                       Defaults to 'no'.</li>
        </ul>

      <b>NOTE</b>:
          This method simulates the entire process of sending a bit, including preparation, potential interception,
          and measurement. It also handles error checking for uninitialized objects that are required in the given case.


      Example Usage:
      ```python
      # Alice sends a 1 without an H gate and Bob receives and measures the qubit without an H gate.
      # The default is that they will not compare their bits and instead will add them to their key
      # if they both made the same choice of H gate or not.
      protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no')


      # Same as above, except Alice and Bob will compare bits instead of adding them to their key.
      protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')
      ```
      """

      self.alice_bit = alice_bit
      self.does_alice_apply_H = does_alice_apply_H
      self.does_bob_apply_H = does_bob_apply_H

      if self.qubit == None:
        print('Error: A qubit object must be defined first.')
        return

      elif self.simulator == None:
        print('Error: A simulator object must be defined first.')
        return


      # Prepare Alice's qubit
      alice_circuit = self.phase_1_circuit()

      if alice_circuit == None:

        applies_H = 'no H'
        if self.does_alice_apply_H == 'yes':
          applies_H = 'an H'

        print('Error: Alice\'s circuit for sending a', self.alice_bit, 'with', applies_H, 'must be defined first.')
        return


      # Eve's interception
      eve_circuit = self.eve_circuit()

      if eve_circuit == None:

        print('Error: Eve\'s interception circuit must be defined first.')
        return

      # Bob's measurement
      bob_circuit = self.phase_2_circuit()

      if bob_circuit == None:

        applies_H = 'no H'
        if self.does_bob_apply_H == 'yes':
          applies_H = 'an H'

        print('Error: Bob\'s circuit for measuring with', applies_H, 'must be defined first.')
        return

      # Combine circuits and run simulation
      bb84_circuit = alice_circuit + eve_circuit + bob_circuit
      results = self.simulator.run(bb84_circuit)
      self.bob_bit = results.measurements['q0'][0][0]

      if self.eve_intercept:
        self.eve_bit = results.measurements.get('eve', [[None]])[0][0]

      # Print results
      print('\033[43m\033[1mATTEMPTED BIT', self.bit_num, '\033[0m\033[0m')

      print('\n\033[32m\033[1mPHASE 1: SENDING\033[0m\033[0m\033[0m')
      print('\033[47m\033[1mAlice (to herself)\033[0m\033[0m: I sent a', self.alice_bit, 'and', 'did not use' if self.does_alice_apply_H == 'no' else 'used', 'an H')

      if self.eve_intercept == 'yes':
        print('EVE INTERCEPTS!')
        print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: I measured a', self.eve_bit, 'and will now send the qubit to Bob')

      print('\n\033[32m\033[1mPHASE 2: RECEIVING\033[0m\033[0m')
      print('\033[47m\033[1mBob (to himself)\033[0m\033[0m: I', 'did not use' if self.does_bob_apply_H == 'no' else 'used', 'an H and measured a', self.bob_bit)

      print('\n\033[32m\033[1mPHASE 3: COMPARING\033[0m\033[0m')
      print('Alice and Bob are comparing choice of H\'s', 'and the bits themselves.' if compare_bits == 'yes' else 'but not the bits themselves.', '\n')
      print('\033[47m\033[1mAlice\033[0m\033[0m: I', 'did not use' if self.does_alice_apply_H == 'no' else 'used', 'an H')
      print('\033[47m\033[1mBob\033[0m\033[0m: I', 'did not use' if self.does_bob_apply_H == 'no' else 'used', 'an H')

      print('')
      if compare_bits == 'yes':
        print('\033[47m\033[1mAlice\033[0m\033[0m: I sent a', self.alice_bit)
        print('\033[47m\033[1mBob\033[0m\033[0m: I measured a', self.bob_bit, '\n')

        if self.does_alice_apply_H == self.does_bob_apply_H:
          if self.alice_bit == self.bob_bit:
            print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: Our bits match, so it doesn\'t seem like Eve is intercepting.')

            if self.eve_intercept == 'yes':
              print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: Mwuhaha, I\'ve gone undetected.')

          else:
            print('\033[91m\033[1mAlice and Bob: Our bits are different, so Eve must have intercepted! Let\'s start over with new keys and a new quantum channel!\033[0m\033[0m')
            self.restart()

        else:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: We made different choices, we should not use this bit.')

      else:
        if self.does_alice_apply_H == self.does_bob_apply_H:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: Great, let\'s add this bit to our keys.')
          self.alice_key += [self.alice_bit]
          self.bob_key += [self.bob_bit]

          if self.eve_intercept:
            self.eve_key += [self.eve_bit]
        else:
          print('\033[47m\033[1mAlice and Bob\033[0m\033[0m: We made different choices, we should not use this bit.')

      print('')
      print('\033[47m\033[1mAlice (to herself)\033[0m\033[0m: My key is now', self.alice_key)
      print('\033[47m\033[1mBob (to himself)\033[0m\033[0m: My key is now', self.bob_key)

      if self.eve_intercept == 'yes':
        print('\033[47m\033[1mEve (to herself)\033[0m\033[0m: My key is now:', self.eve_key)


      print('\nThe circuit used this round:', bb84_circuit)

      self.bit_num += 1
      print('='*75, end='\n\n')

import matplotlib.pyplot as plt

installing cirq...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 787.5 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 6.4 MB/s eta 0

<a name="p1"></a>

---
## **Part 1: Alice and Bob Review**
---

**Run the cell below to fully initialize the BB84 object.**

In [ ]:
# Create the BB84 object
#=======================
my_protocol = BB84()

my_protocol.qubit = cirq.NamedQubit('q0')
my_protocol.simulator = cirq.Simulator()


# Alice Sending
#===============

# 0 without H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_0_no_H_circuit = circuit


# 0 with H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_0_H_circuit = circuit


# 1 without H
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_1_no_H_circuit = circuit

# 1 with H
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_1_H_circuit = circuit



# Bob Receiving
#===============

# without H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_no_H_circuit = circuit


# with H
circuit = cirq.Circuit()

circuit.append(cirq.H(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_H_circuit = circuit

#### **Problem #1.1**

Using the `my_protocol` object created above, perform the following transmissions:

* Alice sends a 0 without an H gate and Bob does not use an H gate. **This has already been provided**.

* Alice sends a 1 without an H gate and Bob does not use an H gate.

* Alice sends a 0 with an H gate and Bob does not use an H gate.

* Alice sends a 1 with an H gate and Bob uses an H gate.

In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'no')

In [ ]:
# COMPLETE THIS CODE

In [ ]:
# COMPLETE THIS CODE

In [ ]:
# COMPLETE THIS CODE

##### **Solution**

In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'no')

ATTEMPTED BIT 1 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0]
Bob (to himself): My key is now [0]

The circuit used this round: q0: ───I───I───I───M───



In [ ]:
my_protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'no')

ATTEMPTED BIT 2 

PHASE 1: SENDING
Alice (to herself): I sent a 1 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 1]
Bob (to himself): My key is now [0, 1]

The circuit used this round: q0: ───X───I───I───M───



In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'no', compare_bits = 'no')

ATTEMPTED BIT 3 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I used an H
Bob: I did not use an H

Alice and Bob: We made different choices, we should not use this bit.

Alice (to herself): My key is now [0, 1]
Bob (to himself): My key is now [0, 1]

The circuit used this round: q0: ───I───H───I───M───



In [ ]:
my_protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bits = 'no')

ATTEMPTED BIT 4 

PHASE 1: SENDING
Alice (to herself): I sent a 1 and used an H

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I used an H
Bob: I used an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 1, 1]
Bob (to himself): My key is now [0, 1, 1]

The circuit used this round: q0: ───X───H───H───M───



#### **Problem #1.2**

Now, send two more transmissions:
*  Alice sends a 0 without an H gate and Bob does not use an H gate and **they compare bits**.

*  Alice sends a 1 without an H gate and Bob does not use an H gate and **they compare bits**.

In [ ]:
# COMPLETE THIS CODE

In [ ]:
# COMPLETE THIS CODE

##### **Solution**

In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')

ATTEMPTED BIT 5 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice: I sent a 0
Bob: I measured a 0 

Alice and Bob: Our bits match, so it doesn't seem like Eve is intercepting.

Alice (to herself): My key is now [0, 1, 1]
Bob (to himself): My key is now [0, 1, 1]

The circuit used this round: q0: ───I───I───I───M───



In [ ]:
my_protocol.send_bit(alice_bit = 1, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')

ATTEMPTED BIT 6 

PHASE 1: SENDING
Alice (to herself): I sent a 1 and did not use an H

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice: I sent a 1
Bob: I measured a 1 

Alice and Bob: Our bits match, so it doesn't seem like Eve is intercepting.

Alice (to herself): My key is now [0, 1, 1]
Bob (to himself): My key is now [0, 1, 1]

The circuit used this round: q0: ───X───I───I───M───



#### **Reflection**

* What changed in the outputs by having Alice and Bob compare their bits?

* Did they detect Eve? Should they have?

<a name="p2"></a>

---
## **Part 2: Eve**
---

In this part, we will implement Eve's interception in BB84.

#### **Problem #2.1**

In the cell below, create a new `BB84` object such that Eve intercepts.

In [ ]:
my_protocol = BB84(# COMPLETE THIS CODE


# Qubit and Simulator
#==================
my_protocol.qubit = cirq.NamedQubit('q0')
my_protocol.simulator = cirq.Simulator()


# Alice's Circuits
#==================
# 0 with no H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_0_no_H_circuit = circuit


# 1 with no H
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_1_no_H_circuit = circuit


# 0 with H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_0_H_circuit = circuit


# 1 with H
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_1_H_circuit = circuit


# Bob's Circuits
#==================
# with no H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_no_H_circuit = circuit

# with H
circuit = cirq.Circuit()

circuit.append(cirq.H(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_H_circuit = circuit

##### **Solution**

In [ ]:
my_protocol = BB84(eve_intercept = 'yes')


# Qubit and Simulator
#==================
my_protocol.qubit = cirq.NamedQubit('q0')
my_protocol.simulator = cirq.Simulator()


# Alice's Circuits
#==================
# 0 with no H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_0_no_H_circuit = circuit


# 1 with no H
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.I(my_protocol.qubit))

my_protocol.alice_send_1_no_H_circuit = circuit


# 0 with H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_0_H_circuit = circuit


# 1 with H
circuit = cirq.Circuit()

circuit.append(cirq.X(my_protocol.qubit))
circuit.append(cirq.H(my_protocol.qubit))

my_protocol.alice_send_1_H_circuit = circuit


# Bob's Circuits
#==================
# with no H
circuit = cirq.Circuit()

circuit.append(cirq.I(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_no_H_circuit = circuit

# with H
circuit = cirq.Circuit()

circuit.append(cirq.H(my_protocol.qubit))
circuit.append(cirq.measure(my_protocol.qubit))

my_protocol.bob_receive_H_circuit = circuit

#### **Problem #2.2**

The only new part we need to define is what circuit Eve will use to intercept. While there are many options, let's keep things simple for now.

<br>

In particular, Eve should only measure the qubit. To ensure her measurement is easy to tell apart from Bob's, set the parameter `key = 'eve'` for the measurement.

In [ ]:
circuit = cirq.Circuit()

# COMPLETE THIS CODE

my_protocol.eve_intercept_circuit = circuit

##### **Solution**

In [ ]:
circuit = cirq.Circuit()

circuit.append(cirq.measure(my_protocol.qubit, key = 'eve'))

my_protocol.eve_intercept_circuit = circuit

#### **Problem #2.3**

Now, send 4 qubits as follows:

* Alice sends a 0 with no H and Bob does not use an H. They do not compare bits.

* Alice sends a 0 with an H and Bob uses an H. They do not compare bits.

* Alice sends a 0 with no H and Bob does not use an H. They do compare bits.

* Alice sends a 0 with an H and Bob uses an H. They do compare bits. (Same as above)

<br>

**NOTE**: The syntax is identical to the case without Eve.

In [ ]:
# COMPLETE THIS CODE

In [ ]:
# COMPLETE THIS CODE

In [ ]:
# COMPLETE THIS CODE

In [ ]:
# COMPLETE THIS CODE

##### **Solution**

In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')

ATTEMPTED BIT 1 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0]
Bob (to himself): My key is now [0]
Eve (to herself): My key is now: [0]

The circuit used this round: q0: ───I───I───M('eve')───I───M───



In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes')

ATTEMPTED BIT 2 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H
EVE INTERCEPTS!
Eve (to herself): I measured a 1 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I used an H
Bob: I used an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0, 0]
Bob (to himself): My key is now [0, 1]
Eve (to herself): My key is now: [0, 1]

The circuit used this round: q0: ───I───H───M('eve')───H───M───



In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bit = 'yes')

ATTEMPTED BIT 3 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice: I sent a 0
Bob: I measured a 0 

Alice and Bob: Our bits match, so it doesn't seem like Eve is intercepting.
Eve (to herself): Mwuhaha, I've gone undetected.

Alice (to herself): My key is now [0, 0]
Bob (to himself): My key is now [0, 1]
Eve (to herself): My key is now: [0, 1]

The circuit used this round: q0: ───I───I───M('eve')───I───M───



In [ ]:
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bit = 'yes')

ATTEMPTED BIT 4 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H
EVE INTERCEPTS!
Eve (to herself): I measured a 1 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I used an H
Bob: I used an H

Alice: I sent a 0
Bob: I measured a 0 

Alice and Bob: Our bits match, so it doesn't seem like Eve is intercepting.
Eve (to herself): Mwuhaha, I've gone undetected.

Alice (to herself): My key is now [0, 0]
Bob (to himself): My key is now [0, 1]
Eve (to herself): My key is now: [0, 1]

The circuit used this round: q0: ───I───H───M('eve')───H───M───



#### **Reflection**

* Before Alice and Bob compared bits, were their keys the same or not? Did they believe they were the same or not?

* Before Eve was caught (if she was caught at all), how did her key compare to Alice and Bob's?

* Did Alice and Bob ever catch Eve?

##### **Solution**

* Their first bit should definitely agree since there was never any superposition involved. However, their second bit *could* have been different without them realizing it due to Eve's interception since the qubit was in a superposition. This potential difference is exactly what Alice and Bob are hoping to exploit to catch Eve intercepting.

* Related to the answer above, the first bit should have been the same for all three of them. However, as soon as Alice uses an H gate, she leaves Eve with a 50/50 chance of getting the same value as Alice. From there, if Bob applies an H gate he has a 50/50 chance of getting the same value as Eve. In this situation, there's no telling whether Eve and/or Bob have the same bit as Alice. Furthermore, if they don't compare bits then none of them will know if their keys are the same or not! This is one way to see just how disruptive BB84 makes Eve's presence.

* The first bit they compare (attempted bit 3) was not in superposition, so Eve could not have caused any noticeable effect with her measurement and therefore she should not have been caught. In the second case (attempted bit 4), the qubit was in superposition so Eve *could* have caused a noticeable effect and therefore been caught.

#### **Problem #2.4**

In the space below, restart the protocol using the `restart()` method and then find a way to send 10 bits such that:
* Alice and Bob compare 5 bits.
* Alice, Bob, and Eve all end up with the same key that is 5 bits long.
* Eve is never detected.


<br>

**NOTE**: There are many possible solutions, but you will have to think about under what conditions Eve can go undetected and everyone can have the same key.

In [ ]:
# COMPLETE THIS CODE

##### **Solution**

In [ ]:
my_protocol.restart()

my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')

my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bit = 'yes')

ATTEMPTED BIT 1 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not use an H

Alice and Bob: Great, let's add this bit to our keys.

Alice (to herself): My key is now [0]
Bob (to himself): My key is now [0]
Eve (to herself): My key is now: [0]

The circuit used this round: q0: ───I───I───M('eve')───I───M───

ATTEMPTED BIT 2 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I did not use an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's but not the bits themselves. 

Alice: I did not use an H
Bob: I did not 

#### **Problem #2.5**

In the space below, restart the protocol using the `restart()` method and then find a way to send 10 bits such that:
* Alice and Bob compare 5 bits.
* Eve is detected as early as possible.


<br>

**NOTE**: There are many possible solutions, but you may get different results each time due to the nature of quantum mechanics!

In [ ]:
# COMPLETE THIS CODE

##### **Solution**

In [ ]:
my_protocol.restart()

my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bit = 'yes')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bit = 'yes')

my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no')

ATTEMPTED BIT 1 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I used an H
Bob: I used an H

Alice: I sent a 0
Bob: I measured a 1 

Alice and Bob: Our bits are different, so Eve must have intercepted! Let's start over with new keys and a new quantum channel!

Alice (to herself): My key is now []
Bob (to himself): My key is now []
Eve (to herself): My key is now: []

The circuit used this round: q0: ───I───H───M('eve')───H───M───

ATTEMPTED BIT 2 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the 

---
#### **Wait for your instructor to continue.**
---

#### **Problem #2.6 [CHALLENGE]**

In the space below, implement an intercept and resend attack while Alice and Bob sends 5 bits that they compare such that:
* For the first bit, redefine the `eve_intercept_circuit` so that Eve applies an H, measures, and applies another H. Then have Alice and Bob send a bit in a way that Alice and Bob will automatically throw out this bit.

* For the second bit, keep Eve's circuit the same as above. Then have Alice and Bob send a bit such that they definitely couldn't notice Eve, despite Alice and Bob comparing bits.

* For the third bit, keep Eve's circuit the same as above. Then have Alice and Bob send a bit such that they *could* notice Eve.

* For the fourth bit, redefine the `eve_intercept_circuit` so that Eve only measures the qubit and sends it (equivalent to "randomly" choosing not to apply H gates). Then have Alice and Bob send a bit such that they definitely couldn't notice Eve, despite Alice and Bob comparing bits.

* For the fifth bit, keep Eve's circuit the same as above. Then have Alice and Bob send a bit such that they *could* notice Eve.

<br>

**NOTE**: It's up to you if they send 0s or 1s in each case, since it doesn't actually affect Eve's ability to go undetected or not.

In [ ]:
my_protocol.restart()

# COMPLETE THIS CODE

##### **Solution**

**NOTE**: It's not necessary to define the two circuits beforehand, but it makes things a little cleaner in practice.

In [ ]:
# Eve's 2 possible circuits in an intercept an resend attack

# Option 1
circuit_1 = cirq.Circuit()
circuit_1.append(cirq.measure(my_protocol.qubit, key = 'eve'))


# Option 2
circuit_2 = cirq.Circuit()

circuit_2.append(cirq.H(my_protocol.qubit))
circuit_2.append(cirq.measure(my_protocol.qubit, key = 'eve'))
circuit_1.append(cirq.H(my_protocol.qubit))

In [ ]:
my_protocol.restart()

# Bit 1
my_protocol.eve_intercept_circuit = circuit_2
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'yes', compare_bits = 'yes')

# Bit 2
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bits = 'yes')

# Bit 3
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')

# Bit 4
my_protocol.eve_intercept_circuit = circuit_1
my_protocol.send_bit(alice_bit = 0 , does_alice_apply_H = 'no', does_bob_apply_H = 'no', compare_bits = 'yes')

# Bit 5
my_protocol.send_bit(alice_bit = 0, does_alice_apply_H = 'yes', does_bob_apply_H = 'yes', compare_bits = 'yes')

ATTEMPTED BIT 1 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and did not use an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 1

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I did not use an H
Bob: I used an H

Alice: I sent a 0
Bob: I measured a 1 

Alice and Bob: We made different choices, we should not use this bit.

Alice (to herself): My key is now []
Bob (to himself): My key is now []
Eve (to herself): My key is now: []

The circuit used this round: q0: ───I───I───H───M('eve')───H───M───

ATTEMPTED BIT 2 

PHASE 1: SENDING
Alice (to herself): I sent a 0 and used an H
EVE INTERCEPTS!
Eve (to herself): I measured a 0 and will now send the qubit to Bob

PHASE 2: RECEIVING
Bob (to himself): I used an H and measured a 0

PHASE 3: COMPARING
Alice and Bob are comparing choice of H's and the bits themselves. 

Alice: I used an H
B

# End of Notebook

---
© 2024 The Coding School, All rights reserved